<a href="https://colab.research.google.com/github/Trantracy/Synthesised-transaction-analysis-at-ANZ/blob/master/ANZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd

df_data = pd.read_excel("/content/drive/My Drive/Colab Notebooks/ANZ/ANZ synthesised transaction dataset.xlsx")

# Overview dataset

In [ ]:
df_data.sample(5)

,status,card_present_flag,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,merchant_code,first_name,balance,date,gender,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,country,customer_id,merchant_long_lat,movement
3115,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,5eb643ae-93ff-4af7-a134-99d6d3aa48be,NaN,Diana,1655.47,2018-08-25,F,26,Robina,QLD,2018-08-25T19:34:11.000+0000,8.87,7cdc397b2d6c4716b6f872c8c8a7fc30,Australia,CUS-2487424745,153.39 -28.08,debit
4545,authorized,1.0,NaN,ACC-1222300524,AUD,151.23 -33.94,SALES-POS,09f289b2-6d59-4325-b710-689496f66c77,NaN,Michael,181.13,2018-09-05,M,38,Williamstown North,VIC,2018-09-05T16:06:26.000+0000,5.04,fe77129bbf1d43c5b59ead41a13b8f76,Australia,CUS-2142601169,144.86 -37.85,debit
8458,posted,NaN,0,ACC-53508546,AUD,150.62 -33.76,PAY/SALARY,NaN,0.0,Kimberly,6566.87,2018-10-04,F,35,NaN,NaN,2018-10-04T17:00:00.000+0000,725.32,afa3917eacd545eeaad9d925cec54e8d,Australia,CUS-72755508,NaN,credit
4752,posted,NaN,NaN,ACC-1710017148,AUD,150.82 -34.01,PAYMENT,NaN,NaN,Michelle,8975.12,2018-09-07,F,19,NaN,NaN,2018-09-07T04:00:00.000+0000,40.00,95fea6659926486bb9b7ac6a4b3471fb,Australia,CUS-883482547,NaN,debit
6719,posted,NaN,NaN,ACC-1652235822,AUD,144.89 -37.81,PAYMENT,NaN,NaN,Renee,19755.62,2018-09-21,F,38,NaN,NaN,2018-09-21T18:00:00.000+0000,66.00,85aa85e5799d4f0098ff7b17bcc45bc5,Australia,CUS-2206365095,NaN,debit


In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12043 entries, 0 to 12042
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   status             12043 non-null  object        
 1   card_present_flag  7717 non-null   float64       
 2   bpay_biller_code   885 non-null    object        
 3   account            12043 non-null  object        
 4   currency           12043 non-null  object        
 5   long_lat           12043 non-null  object        
 6   txn_description    12043 non-null  object        
 7   merchant_id        7717 non-null   object        
 8   merchant_code      883 non-null    float64       
 9   first_name         12043 non-null  object        
 10  balance            12043 non-null  float64       
 11  date               12043 non-null  datetime64[ns]
 12  gender             12043 non-null  object        
 13  age                12043 non-null  int64         
 14  mercha

In [ ]:
# Summarizing Data
df_data.describe()

,card_present_flag,merchant_code,balance,age,amount
count,7717.000000,883.0,12043.000000,12043.000000,12043.000000
mean,0.802644,0.0,14704.195553,30.582330,187.933588
std,0.398029,0.0,31503.722652,10.046343,592.599934
min,0.000000,0.0,0.240000,18.000000,0.100000
25%,1.000000,0.0,3158.585000,22.000000,16.000000
50%,1.000000,0.0,6432.010000,28.000000,29.000000
75%,1.000000,0.0,12465.945000,38.000000,53.655000
max,1.000000,0.0,267128.520000,78.000000,8835.980000


In [ ]:
df_data.describe(include='object')

,status,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,first_name,gender,merchant_suburb,merchant_state,extraction,transaction_id,country,customer_id,merchant_long_lat,movement
count,12043,885,12043,12043,12043,12043,7717,12043,12043,7717,7717,12043,12043,12043,12043,7717,12043
unique,2,3,100,1,100,6,5725,80,2,1609,8,9442,12043,1,100,2703,2
top,authorized,0,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,106e1272-44ab-4dcb-a438-dd98e0071e51,Michael,M,Melbourne,NSW,2018-09-03T11:00:00.000+0000,54c7f107e7b64e0084173a8d9a235151,Australia,CUS-2487424745,151.21 -33.87,debit
freq,7717,883,578,12043,578,3934,14,746,6285,255,2169,10,1,12043,578,145,11160


# Explore the data:

## Checking how many unique customers in the dataset:

In [ ]:
#len(df_data['account'].value_counts().index)
df_data['account'].nunique()

100

## Check missing data:

In [ ]:
df_data.isnull().sum()

status                   0
card_present_flag     4326
bpay_biller_code     11158
account                  0
currency                 0
long_lat                 0
txn_description          0
merchant_id           4326
merchant_code        11160
first_name               0
balance                  0
date                     0
gender                   0
age                      0
merchant_suburb       4326
merchant_state        4326
extraction               0
amount                   0
transaction_id           0
country                  0
customer_id              0
merchant_long_lat     4326
movement                 0
dtype: int64

# Dropping the irrelevant features and add some feature which is helpful for analysis:


In [ ]:
# Drop irrelevant columns:
df = df_data.drop(columns=['status','card_present_flag','country','merchant_id','currency','merchant_code','account','bpay_biller_code'])
# Convert the object time to datattime: 
df["date"] = pd.to_datetime(df["date"])
df["extraction"] = pd.to_datetime(df["extraction"])
# Adding a hour column in the dataframe
df['Transaction_time'] = df['extraction'].dt.hour
# add a weekday column in dataset:
df['weekday'] = df['date'].dt.day_name()
df.head(3)

,long_lat,txn_description,first_name,balance,date,gender,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,customer_id,merchant_long_lat,movement,Transaction_time,weekday
0,153.41 -27.95,POS,Diana,35.39,2018-08-01,F,26,Ashmore,QLD,2018-08-01 01:01:15+00:00,16.25,a623070bfead4541a6b0fff8a09e706c,CUS-2487424745,153.38 -27.99,debit,1,Wednesday
1,153.41 -27.95,SALES-POS,Diana,21.20,2018-08-01,F,26,Sydney,NSW,2018-08-01 01:13:45+00:00,14.19,13270a2a902145da9db4c951e04b51b9,CUS-2487424745,151.21 -33.87,debit,1,Wednesday
2,151.23 -33.94,POS,Michael,5.71,2018-08-01,M,38,Sydney,NSW,2018-08-01 01:26:15+00:00,6.42,feb79e7ecd7048a5a36ec889d1a94270,CUS-2142601169,151.21 -33.87,debit,1,Wednesday


## The number of transactions and value per day/week/month:

### The number of transaction By the time in day: 

In [ ]:
hr_transactions = df.groupby(df['Transaction_time']).agg({'transaction_id': [('Number of transactions','count')]})
hr_transactions.columns = hr_transactions.columns.droplevel()
hr_transactions = hr_transactions.sort_values(by='Number of transactions', ascending= False)
pd.set_option("max_rows", None)
hr_transactions

,Number of transactions
Transaction_time,
9,1106
11,941
17,815
14,757
21,741
15,728
10,615
20,598
13,558


#### By day:

In [ ]:
# By day:
no_transactions = df.groupby(df['date']).agg({'transaction_id': [('Number of transactions','count')],'amount':[('Total Amount','sum')]})
no_transactions.columns = no_transactions.columns.droplevel()
no_transactions = no_transactions.sort_values(by='Total Amount', ascending= False)
pd.set_option("max_rows", None)
no_transactions

,Number of transactions,Total Amount
date,,
2018-10-15,112,51943.08
2018-10-19,158,51252.75
2018-09-14,161,48736.97
2018-08-20,97,48240.34
2018-10-26,157,46451.05
2018-08-06,99,42288.23
2018-09-17,112,42167.38
2018-08-17,172,41906.21
2018-09-26,149,41713.17


In [ ]:
# the number of days: 
no_transactions.shape[0]

91

In [ ]:
# The average volume per day: 
round(no_transactions['Number of transactions'].mean(),2)

132.34

In [ ]:
# The avarage value per day: 
round(no_transactions['Total Amount'].mean(),2)

24871.25

### by month:

In [ ]:
# By month:
mo_transactions = df.groupby(df['date'].dt.month).agg({'transaction_id': [('Number of transactions','count')],'amount':[('Total Amount','sum')]})
mo_transactions.columns = mo_transactions.columns.droplevel()
mo_transactions = mo_transactions.sort_values(by='Total Amount', ascending= False)
pd.set_option("max_rows", None)
mo_transactions

,Number of transactions,Total Amount
date,,
10,4087,802798.47
9,4013,730550.21
8,3943,729935.52


### By week:

In [ ]:
# No transaction by weekday and type of transaction:
w_transactions = df.groupby(['txn_description','weekday']).agg({'transaction_id': [('Number of transactions','count')]})
w_transactions.columns = w_transactions.columns.droplevel()
w_transactions = w_transactions.sort_values(by='Number of transactions', ascending= False)
w_transactions

Number of transactions
txn_description weekday                          
SALES-POS       Saturday                      685
                Friday                        662
POS             Friday                        655
                Saturday                      652
SALES-POS       Wednesday                     649
POS             Wednesday                     602
SALES-POS       Sunday                        593
POS             Thursday                      573
                Sunday                        533
SALES-POS       Thursday                      530
PAYMENT         Friday                        470
                Thursday                      442
                Wednesday                     441
SALES-POS       Tuesday                       427
POS             Tuesday                       414
SALES-POS       Monday                        388
POS             Monday                        354
PAYMENT         Tuesday                       345
                Monday                        311
                Sunday                        297
                Saturday                      294
PAY/SALARY      Monday                        207
                Friday                        201
                Wednesday                     172
INTER BANK      Wednesday                     169
PAY/SALARY      Tuesday                       160
                Thursday                      143
INTER BANK      Tuesday                       123
                Sunday                        122
                Monday                         99
                Thursday                       87
                Friday                         80
                Saturday                       62
PHONE BANK      Wednesday                      30
                Thursday                       26
                Tuesday                        18
                Saturday                       16
                Sunday                          5
                Friday                          5
                Monday                          1

# Total Numbers of transactions and value made by each customers:

In [ ]:
each_cus = df.groupby(['first_name','customer_id']).agg({'transaction_id':[('Number of transactions','count')],'amount':[('Total Amount','sum')]})
each_cus.columns = each_cus.columns.droplevel()
each_cus = each_cus.sort_values(by='Total Amount', ascending= False)
each_cus

,,Number of transactions,Total Amount
first_name,customer_id,,
Kenneth,CUS-2738291516,80,45409.16
Ricky,CUS-3142625864,163,42688.30
Tim,CUS-1816693151,78,40215.54
Linda,CUS-2155701614,101,37943.79
Kenneth,CUS-261674136,134,36786.13
Michelle,CUS-883482547,178,36639.41
Sandra,CUS-4142663097,174,36588.25
Natasha,CUS-527400765,151,36543.61
Jessica,CUS-1196156254,245,36050.72


In [ ]:
# The average volume per customer: 
round(each_cus['Number of transactions'].mean(),2)

120.43

In [ ]:
# The avarage value per customer: 
round(each_cus['Total Amount'].mean(),2)

22632.84

In [ ]:
# top 10 customers:
each_cus.nlargest(10,'Total Amount')

,,Number of transactions,Total Amount
first_name,customer_id,,
Kenneth,CUS-2738291516,80,45409.16
Ricky,CUS-3142625864,163,42688.30
Tim,CUS-1816693151,78,40215.54
Linda,CUS-2155701614,101,37943.79
Kenneth,CUS-261674136,134,36786.13
Michelle,CUS-883482547,178,36639.41
Sandra,CUS-4142663097,174,36588.25
Natasha,CUS-527400765,151,36543.61
Jessica,CUS-1196156254,245,36050.72


# The transaction type and the total value of each type: 

In [ ]:
type_trans = df.groupby(['txn_description', 'movement']).agg({'transaction_id':[('Number of transactions','count')],'amount':[('Total Amount','sum')]})
type_trans.columns = type_trans.columns.droplevel()
type_trans = type_trans.sort_values(by='Total Amount', ascending= False)
type_trans

,,Number of transactions,Total Amount
txn_description,movement,,
PAY/SALARY,credit,883,1676576.85
PAYMENT,debit,2600,201794.00
SALES-POS,debit,3934,157005.11
POS,debit,3783,152861.24
INTER BANK,debit,742,64331.00
PHONE BANK,debit,101,10716.00


## Customer analysis with gender and age:

In [ ]:
# average age of each gender:
gen_age = df.groupby(['gender']).agg({'gender':[('Total','count')],'age':[('Average_age','mean')]})
gen_age.columns = gen_age.columns.droplevel()
gen_age

,Total,Average_age
gender,,
F,5758,30.460750
M,6285,30.693715


In [ ]:
# Total amount spent regarding to the age to the age under 24
age24 = df[df['age']<=24].groupby(['txn_description']).agg({'amount':[('Total amount','sum')]})
age24

,amount
,Total amount
txn_description,
INTER BANK,27501.00
PAY/SALARY,558016.92
PAYMENT,63913.00
PHONE BANK,1874.00
POS,46429.87
SALES-POS,49311.02


In [ ]:
# Total amount spent regarding to the age between 25 and 40
age24_40 = df[(df['age']> 24) & (df['age'] <= 40)].groupby(['txn_description']).agg({'amount':[('Total amount','sum')]})
age24_40

,amount
,Total amount
txn_description,
INTER BANK,23653.00
PAY/SALARY,833793.81
PAYMENT,99789.00
PHONE BANK,5713.00
POS,81658.79
SALES-POS,87649.74


In [ ]:
# Total amount spent regarding to the age over 40
age40 = df[df['age'] > 40].groupby(['txn_description']).agg({'amount':[('Total amount','sum')]})
age40

,amount
,Total amount
txn_description,
INTER BANK,13177.00
PAY/SALARY,284766.12
PAYMENT,38092.00
PHONE BANK,3129.00
POS,24772.58
SALES-POS,20044.35


# Location analysis:

### Number of transaction by location:

In [ ]:
loc_trans = df.groupby(['merchant_suburb','merchant_state']).agg({'transaction_id':[('Number of transaction','count')]})
loc_trans.columns = loc_trans.columns.droplevel()
loc_trans

In [ ]:
df.sample(5)

,long_lat,txn_description,first_name,balance,date,gender,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,customer_id,merchant_long_lat,movement,Transaction_time,weekday
4319,145.04 -37.92,POS,Derek,2117.12,2018-09-04,M,20,Melbourne,VIC,2018-09-04 01:02:56+00:00,9.33,e6652cf8bc4d4c069da7961832beca90,CUS-3325710106,144.96 -37.82,debit,1,Tuesday
6838,143.88 -37.66,SALES-POS,Alexander,16580.30,2018-09-22,M,38,Roxburgh Park,VIC,2018-09-22 15:12:43+00:00,28.75,5678237a0a1e494b860524814be28f25,CUS-1478398256,144.93 -37.64,debit,15,Saturday
2244,147.31 -42.88,POS,Mary,1486.82,2018-08-19,F,64,North Hobart,TAS,2018-08-19 03:42:51+00:00,34.60,5f90fa93db0148549f7b7df7c1cff6ea,CUS-1654129794,147.32 -42.87,debit,3,Sunday
11876,144.79 -37.73,PAYMENT,Tyler,12459.91,2018-10-30,M,44,NaN,NaN,2018-10-30 17:00:00+00:00,27.00,cabbe67f8c4f440c8a7b7235d4032ca4,CUS-1892177589,NaN,debit,17,Tuesday
5357,147.63 -22.84,PAYMENT,James,10529.38,2018-09-11,M,28,NaN,NaN,2018-09-11 21:00:00+00:00,181.00,828387b77211478e9883f8487252131c,CUS-4023861240,NaN,debit,21,Tuesday


In [ ]:
# Top 10 suburb	locations with the largest transactions:
loc_trans.nlargest(10,'Number of transaction')

,,Number of transaction
merchant_suburb,merchant_state,
Melbourne,VIC,255
Sydney,NSW,233
Southport,QLD,82
Brisbane City,QLD,79
Chatswood,NSW,55
Perth,WA,45
Adelaide,SA,44
Broadbeach,QLD,41
Mount Gambier,SA,41
